In [22]:
import xarray as xr
import glob
import os

In [62]:
platFilenames = glob.glob('/storage/ifremer-platform-meta/*/*/*_meta.nc')
print(len(platFilenames))

15267


In [61]:
for filename in platFilenames[0:10]:
    plat = xr.open_dataset('2900201_meta.nc')
    [x.strip(' ') for x in plat.variables['PARAMETER_SENSOR'].data.astype(str)]
    plat.variables['PARAMETER_RESOLUTION'].data.astype(float).tolist()
    pn = int(plat.variables['PLATFORM_NUMBER'].data)

In [37]:
plat.info()

xarray.Dataset {
dimensions:
	N_CONFIG_PARAM = 7 ;
	N_LAUNCH_CONFIG_PARAM = 7 ;
	N_MISSIONS = 1 ;
	N_PARAM = 3 ;
	N_POSITIONING_SYSTEM = 1 ;
	N_SENSOR = 3 ;
	N_TRANS_SYSTEM = 1 ;

variables:
	object DATA_TYPE() ;
		DATA_TYPE:long_name = Data type ;
		DATA_TYPE:conventions = Argo reference table 1 ;
	object FORMAT_VERSION() ;
		FORMAT_VERSION:long_name = File format version ;
	object HANDBOOK_VERSION() ;
		HANDBOOK_VERSION:long_name = Data handbook version ;
	object DATE_CREATION() ;
		DATE_CREATION:long_name = Date of file creation ;
		DATE_CREATION:conventions = YYYYMMDDHHMISS ;
	object DATE_UPDATE() ;
		DATE_UPDATE:long_name = Date of update of this file ;
		DATE_UPDATE:conventions = YYYYMMDDHHMISS ;
	object PLATFORM_NUMBER() ;
		PLATFORM_NUMBER:long_name = Float unique identifier ;
		PLATFORM_NUMBER:conventions = WMO float identifier : A9IIIII ;
	object PTT() ;
		PTT:long_name = Transmission identifier (ARGOS, ORBCOMM, etc.) ;
	object TRANS_SYSTEM(N_TRANS_SYSTEM) ;
		TRANS_SYSTEM:lo

In [19]:
[x.strip(' ') for x in plat.variables['PARAMETER_SENSOR'].data.astype(str)]

['CTD_PRES', 'CTD_TEMP', 'CTD_CNDC']

In [20]:
plat.variables['PARAMETER_RESOLUTION'].data.astype(float).tolist()

[0.1, 0.001, 0.001]

In [59]:
pn = int(plat.variables['PLATFORM_NUMBER'].data)

In [60]:
pn

2900201